In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,precision_recall_fscore_support

from sklearn.linear_model import SGDRegressor

import torch
import torch.nn as nn

# Reproducibility
RANDOM_STATE=42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

# Data Prep
data=load_breast_cancer()
X=data.data.astype(np.float64)
y=data.target.astype(np.int64)

# Map {0,1} -> {-1,+1} for ADALINE
y=np.where(y==0,-1,1)

X_train,X_test,y_train,y_test=train_test_split(
    X,y,test_size=0.25,random_state=RANDOM_STATE,stratify=y
)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# Helpers
def metrics_report(y_true,y_pred,pos_label=1,name="Model"):
    from sklearn.metrics import accuracy_score,confusion_matrix,precision_recall_fscore_support
    import numpy as np
    import matplotlib.pyplot as plt

    acc=accuracy_score(y_true,y_pred)
    cm=confusion_matrix(y_true,y_pred,labels=[1,-1])
    prec,rec,f1,_=precision_recall_fscore_support(
        y_true,y_pred,average="binary",pos_label=pos_label,zero_division=0
    )

    print(f"\n{name} — Test Metrics")
    print(f"Accuracy:{acc:.4f}")
    print(f"Precision:{prec:.4f} Recall:{rec:.4f} F1:{f1:.4f}")
    print("Confusion Matrix (rows=[+1,-1], cols=[+1,-1]):")
    print(cm)

    fig,ax=plt.subplots(figsize=(5,4))
    im=ax.imshow(cm,interpolation="nearest")
    plt.colorbar(im,ax=ax,fraction=0.046,pad=0.04)

    class_names=["+1","-1"]
    ax.set(
        xticks=np.arange(len(class_names)),
        yticks=np.arange(len(class_names)),
        xticklabels=class_names,
        yticklabels=class_names,
        xlabel="Predicted label",
        ylabel="True label",
        title=f"{name} — Confusion Matrix"
    )

    thresh=cm.max()/2.0 if cm.size else 0.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j,i,format(cm[i,j],"d"),
                ha="center",va="center",
                color="white" if cm[i,j]>thresh else "black",
                fontsize=12
            )

    plt.tight_layout()
    plt.show()

    return {"accuracy":acc,"precision":prec,"recall":rec,"f1":f1,"cm":cm}


def plot_sse(histories,title):
    plt.figure(figsize=(7,5))
    for label,sse in histories.items():
        plt.plot(range(1,len(sse)+1),sse,label=str(label))
    plt.xlabel("Epoch")
    plt.ylabel("SSE")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

# SECTION 3 — AUTOMATIC (scikit-learn via SGDRegressor)
# We drive epochs ourselves with partial_fit to record SSE.
def train_sgd_adaline(Xtr, ytr, lr=0.01, epochs=100, random_state=0):
    model=SGDRegressor(
        loss="squared_error",
        penalty="l2",
        alpha=1e-4,
        learning_rate="constant",
        eta0=lr,
        fit_intercept=True,
        random_state=random_state,
        max_iter=1,
        tol=None
    )
    sse_hist=[]
    for _ in range(epochs):
        model.partial_fit(Xtr,ytr)
        pred_tr=model.predict(Xtr)
        errors=ytr-pred_tr
        sse_hist.append(float(np.sum(errors**2)))
    return model,sse_hist

# CLO-1: Train + SSE vs Epochs (Automatic)
auto_lr=0.01
auto_epochs=100
sgd_model,sgd_hist=train_sgd_adaline(X_train,y_train,lr=auto_lr,epochs=auto_epochs,random_state=RANDOM_STATE)

plot_sse({"SSE":sgd_hist},
         f"ADALINE (sklearn SGDRegressor) — SSE vs Epochs (η={auto_lr})")

# CLO-2: Metrics on Test (Automatic)
y_pred_auto=np.where(sgd_model.predict(X_test)>=0.0,1,-1)
auto_metrics=metrics_report(y_test,y_pred_auto,name=f"ADALINE Automatic (η={auto_lr})")

# CLO-3: Learning Rate Experiments (Automatic)
lrs=[0.001,0.01,0.1,1.0]

hist_auto={}
for lr in lrs:
    mdl,sse_hist=train_sgd_adaline(X_train,y_train,lr=lr,epochs=auto_epochs,random_state=RANDOM_STATE)
    hist_auto[f"η={lr}"]=sse_hist

plot_sse(hist_auto,"ADALINE (sklearn) — SSE vs Epochs for Learning Rates")

# Summary Table (optional)
summary=pd.DataFrame([
    {"Model":f"Manual (η={manual_lr})",**{k:v for k,v in manual_metrics.items() if k!='cm'}},
    {"Model":f"PyTorch (η={torch_lr})",**{k:v for k,v in torch_metrics.items() if k!='cm'}},
    {"Model":f"sklearn (η={auto_lr})",**{k:v for k,v in auto_metrics.items() if k!='cm'}},
])
print("\nSummary (Test)")
print(summary.to_string(index=False))

